BDS^3
=====

## Quantitative microscopy
#### Reconstruction of proteins' biophysical properties with live-cell imaging data
---
_Borys Olifirov, 07.2023, Uzhhorod_

In [ ]:
import os
import sys

import matplotlib.pyplot as plt
import numpy as np
from numpy import ma

from skimage import io
from skimage import filters
from skimage import measure
from skimage import morphology

from scipy.ndimage import measurements
from scipy import signal

## 1. Biophysical properties of calcium ions binding process

## 2. Multidimensional arrays & structure of microscopic data 

## 3. Filters, masks and morphology operations

## 4. Cell detection and segmentation

## 5. 

## 6.

In [ ]:
img_raw = io.imread('post_rgeco_mov_cor.tif')
img_bad = io.imread('post_rgeco.tif')

plt.figure(figsize=(10,10))
plt.imshow(np.max(img_bad, axis=0), cmap='jet')

plt.figure(figsize=(10,10))
plt.imshow(np.max(img_raw, axis=0), cmap='jet')

img = np.array([filters.gaussian(frame) for frame in img_raw])

print(img.shape)

In [ ]:
th_y = filters.threshold_yen(np.max(img, axis=0))
th_o = filters.threshold_otsu(np.max(img, axis=0))
th_l = filters.threshold_li(np.max(img, axis=0))

print(th_y, th_o, th_l)


img_max = np.max(img, axis=0)
plt.figure(figsize=(10,10))
plt.imshow(img_max, cmap='jet')

mask = img_max > th_y
crop_mask = mask[100:200, 100:200]
plt.figure(figsize=(10,10))
plt.imshow(mask)

label_mask = measure.label(mask)
plt.figure(figsize=(10,10))
plt.imshow(label_mask, cmap='jet')

In [ ]:
area_dict = {}
area_th = 150
for label_number in range(1, np.max(label_mask)):
    one_label_mask = label_mask == label_number
    one_label_area = np.sum(one_label_mask)
    
    if one_label_area >= area_th:
        label_area_dict = {label_number:one_label_area}
        area_dict.update(label_area_dict)

fin_mask = np.zeros_like(label_mask)
for lable_key in area_dict.keys():
     key_mask = label_mask == lable_key
     fin_mask = fin_mask + key_mask

fin_raw_mask = fin_mask
fin_mask = morphology.dilation(fin_mask, footprint=morphology.disk(6))
# plt.imshow(fin_raw_mask)
plt.figure(figsize=(10,10))
plt.imshow(fin_mask)


fin_mask = measure.label(fin_mask)
plt.figure(figsize=(10,10))
plt.imshow(fin_mask)

#### Watershed segmentation

In [ ]:
from scipy import ndimage as ndi

from skimage.segmentation import watershed
from skimage.feature import peak_local_max

distance = ndi.distance_transform_cdt(fin_mask, metric='taxicab')
plt.figure(figsize=(10,10))
plt.imshow(distance)

# ndi.distance_transform_cdt, metric='taxicab'

coords = peak_local_max(distance, min_distance=15, footprint=morphology.disk(15), labels=fin_mask)
mask = np.zeros(distance.shape, dtype=bool)
mask[tuple(coords.T)] = True
markers, _ = ndi.label(mask)
labels = watershed(-distance, markers, mask=fin_mask)

plt.figure(figsize=(10,10))
plt.imshow(labels)

#### Profiles extration

In [ ]:
# profile = [np.mean(ma.masked_where(-fin_mask, frame)) for frame in img_raw]

plt.figure(figsize=(20, 8))

prof_arr = []
i = 0
for label in range(1, np.max(labels)+1):
    mask = labels == label
    prof = [np.mean(ma.masked_where(~mask, frame)) for frame in img_raw]
    prof_arr.append(prof)
    plt.plot(prof, label=label)
    i += 200

plt.legend()
plt.show()

prof_arr = np.asarray(prof_arr)

print(prof_arr.shape)


#### Demo labeling

In [ ]:
demo_mask = np.array([[1, 0, 1, 0],
                      [0, 0, 1, 1],
                      [0, 1, 0, 0],
                      [1, 0, 1, 0]])

label_demo_mask = measure.label(demo_mask, connectivity=1)

print(demo_mask)
print(label_demo_mask)
plt.imshow(label_demo_mask)

# Profile an

#### dF/F calc

In [ ]:
dF_prof_arr = []
for prof in prof_arr:
    F_0 = np.mean(prof[0:50])
    dF_prof = (prof - F_0)/F_0
    dF_prof_arr.append(dF_prof)

dF_prof_arr = np.asarray(dF_prof_arr)
plt.figure(figsize=(20, 8))
for dF_p in dF_prof_arr:
    plt.plot(dF_p)
plt.show()    
    
    

#### Peaks detection

In [ ]:
one_profile = dF_prof_arr[2]

peaks, properties = signal.find_peaks(one_profile,
                                      height=0.2,
                                      distance=5,
                                      wlen=30,
                                      prominence=0.15,
                                      rel_height=0.5,
                                      width=1)

print(peaks)
print(properties.keys())
print(properties)

In [ ]:

plt.figure(figsize=(20, 8))
plt.plot(one_profile)
plt.plot(peaks, one_profile[peaks], 'x')
